In [1]:
import win32com.client
import os
import pandas as pd
from datetime import datetime, date
from tqdm import tqdm

In [4]:
SapGuiAuto  = win32com.client.GetObject("SAPGUI")
application = SapGuiAuto.GetScriptingEngine
connection = application.Children(0)
session    = connection.Children(0)

In [5]:
df = pd.read_excel("InsertMaterialNote.xlsx", dtype=str)
df.head(9)




,MATNR,WERKS,OBS
0,80036007,1001,ALTERAR P/DEBITO DIRETO
1,80047404,1001,ALTERAR P/DEBITO DIRETO
2,80038966,1001,ALTERAR P/DEBITO DIRETO
3,80037770,1001,MANTER DEBITO DIRETO
4,80035742,1001,MANTER DEBITO DIRETO
5,80035329,1001,MANTER DEBITO DIRETO
6,80035712,1001,MANTER DEBITO DIRETO
7,80035711,1001,MANTER DEBITO DIRETO
8,80054120,1001,OBSOLETAR/VENDA


In [6]:
df["OBS"].unique()

array(['ALTERAR P/DEBITO DIRETO', 'MANTER DEBITO DIRETO',
       'OBSOLETAR/VENDA', 'ALTERAR P/ESTOQUE', 'MANTER ESTOQUE'],
      dtype=object)

In [7]:
#Abrir tela MM02 (Modificar Material)
session.findById("wnd[0]/tbar[0]/okcd").text = "/NMM02"
session.findById("wnd[0]").sendVKey(0)

############################################################
#<<-------->> Atenção Informar N° SE Atualizado <<-------->>
n_se = "CAD.AP.2024.02001"
#<<------------------------------------------------------->>
############################################################

data = date.today()
data_atual = f'{"%02d"%data.day}/{"%02d"%data.month}/{data.year}'
dt_rev = f'{"%02d"%data.day}.{"%02d"%data.month}.{data.year}'
dt_rev


'27.09.2024'

In [8]:
for sku, cen, obs in tqdm(df.values):
    nota_mrp = f"REANÁLISE OBS: {dt_rev} - {n_se} - {obs}"
    #Informar o SKU
    session.findById("wnd[0]/usr/ctxtRMMG1-MATNR").text = sku #"80059382"
    session.findById("wnd[0]").sendVKey(0)

    user = os.getlogin().upper()
    errouser = f"Dados de grupo de empresas do material {sku} bloqueados pelo usuário {user}"
    sbar = session.findById("wnd[0]/sbar").text        
    while sbar == errouser: 
        session.findById("wnd[0]/tbar[0]/btn[15]").press()       
        session.findById("wnd[0]/tbar[0]/okcd").text = "/NMM02"
        session.findById("wnd[0]").sendVKey(0)
        #Código do material   
        session.findById("wnd[0]/usr/ctxtRMMG1-MATNR").text = sku
        session.findById("wnd[0]/tbar[1]/btn[5]").press() 
        sbar = session.findById("wnd[0]/sbar").text

    #Selecionar as Visões
    session.findById("wnd[1]/tbar[0]/btn[19]").press()        

    l = 4
    while session.findById(f"wnd[1]/usr/tblSAPLMGMMTC_VIEW/txtMSICHTAUSW-DYTXT[0,{l}]").text != "MRP 4":
        l = l + 1            

    if session.findById(f"wnd[1]/usr/tblSAPLMGMMTC_VIEW/txtMSICHTAUSW-DYTXT[0,{l}]").text == "MRP 4":
        session.findById(f"wnd[1]/usr/tblSAPLMGMMTC_VIEW").getAbsoluteRow(l).Selected = True
    session.findById("wnd[1]/tbar[0]/btn[6]").press()
    session.findById("wnd[1]/usr/ctxtRMMG1-WERKS").text = cen #"1005"
    session.findById("wnd[1]/usr/ctxtRMMG1-LGORT").text = ""

    session.findById("wnd[1]/tbar[0]/btn[0]").press()
    #Incluir Nota Material
    session.findById("wnd[0]/usr/tabsTABSPR1/tabpSP15/ssubTABFRA1:SAPLMGMM:2000/subSUB5:SAPLMGD1:2503/btnMARC_NOTE").press()

    if session.findById("wnd[0]/usr/tblSAPLSTXXEDITAREA/txtRSTXT-TXLINE[2,1]").text != "":
        session.findById("wnd[0]/usr/tblSAPLSTXXEDITAREA/ctxtRSTXT-TXPARGRAPH[0,1]").setFocus()
        session.findById("wnd[0]").sendVKey(0)
        session.findById("wnd[0]/usr/tblSAPLSTXXEDITAREA/txtRSTXT-TXLINE[2,1]").text = nota_mrp

    else:
        session.findById("wnd[0]/usr/tblSAPLSTXXEDITAREA/txtRSTXT-TXLINE[2,1]").text = nota_mrp

    session.findById("wnd[0]/tbar[0]/btn[11]").press()
    session.findById("wnd[0]/tbar[0]/btn[3]").press()
    session.findById("wnd[0]").sendVKey(0)

    #Finalizar processo
    session.findById("wnd[1]/usr/btnSPOP-OPTION1").press()
    
session.findById("wnd[0]/tbar[0]/btn[15]").press()


100%|██████████| 40/40 [01:17<00:00,  1.93s/it]
